In [1]:
import pandas as pd
import numpy as np
import glob
import os
#import datacompy as dc
#import fnmatch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# merging the files
path = r'/content/drive/Shared drives/DS4A_2021/Team74/data/raw' # path varaiable for source location 
all_files = glob.glob(path + "/*.csv") #returns list of all joined files 
dtypes = {'jobsreported': np.int64,'originatinglenderlocationid': np.int64,'sbaofficecode': np.int64,'servicinglenderlocationid': np.int64} #Dtype dict
li = []   

for filename in all_files:
    df_file = pd.read_csv(filename,dtype=dtypes, index_col=None, header=0)
    li.append(df_file)

df = pd.concat(li, axis=0, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.head(1)

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
0,5375617707,05/01/2020,101.0,PPP,NOT AVAILABLE,NaN,NaN,NaN,NaN,NaN,Exemption 4,24,100,148440.0,148440.0,0.0,NaN,9551.0,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,NaN,NaN,NaN,NaN,NaN,12.0,339114.0,Unanswered,Unknown/NotStated,NaN,148440.0,NaN,NaN,NaN,NaN,NaN,NaN,9551.0,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,NaN


In [ ]:
# write merged dataframe to drive
df.to_csv (r'/content/drive/Shared drives/DS4A_2021/Team74/data/Stage/loan_files_stage.csv', index = None, header=True) 

In [3]:
# read merged file
df = pd.read_csv('/content/drive/Shared drives/DS4A_2021/Team74/data/Stage/loan_files_stage.csv')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.shape

(10697219, 51)

In [8]:
df.dtypes


LoanNumber                       int64
DateApproved                    object
SBAOfficeCode                  float64
ProcessingMethod                object
BorrowerName                    object
BorrowerAddress                 object
BorrowerCity                    object
BorrowerState                   object
BorrowerZip                     object
LoanStatusDate                  object
LoanStatus                      object
Term                             int64
SBAGuarantyPercentage            int64
InitialApprovalAmount          float64
CurrentApprovalAmount          float64
UndisbursedAmount              float64
FranchiseName                   object
ServicingLenderLocationID      float64
ServicingLenderName             object
ServicingLenderAddress          object
ServicingLenderCity             object
ServicingLenderState            object
ServicingLenderZip              object
RuralUrbanIndicator             object
HubzoneIndicator                object
LMIIndicator             

In [5]:
# Random Sample the sba mereged loan file
sample_df= df.sample(n=100000)

In [6]:
#Remove commas from the column 
sample_df['BorrowerName'] = sample_df['BorrowerName'].str.replace(",","")
#strip leading and trailing space
sample_df['BorrowerName'] = sample_df['BorrowerName'].str.strip()
#Applying uppercase to a column
sample_df['BorrowerName'] = sample_df['BorrowerName'].str.upper()
#Remove commas from all other column
sample_df = sample_df.replace(',','', regex=True)

In [7]:
#Drop rows containing empty borrowerName and CurrentApprovalAmount
sample_df['CurrentApprovalAmount'].replace('', np.nan, inplace=True)
sample_df['BorrowerName'].replace('', np.nan, inplace=True)
sample_df.dropna(subset=['CurrentApprovalAmount','BorrowerName'], inplace=True)

In [8]:
#replace N/A and Unanswered win nan in each column
sample_df = sample_df.replace('Unanswered',np.nan, regex=True)
sample_df = sample_df.replace('N/A',np.nan, regex=True)
sample_df = sample_df.replace('Unknown/NotStated',np.nan, regex=True)
sample_df = sample_df.replace('N/A',np.nan, regex=True)
sample_df['NonProfit'] = sample_df['NonProfit'].replace(np.nan, 'N')



In [9]:
#Convert data types
sample_df['DateApproved'] = pd.to_datetime(sample_df['DateApproved'])
sample_df['LoanStatusDate'] = pd.to_datetime(sample_df['LoanStatusDate']) 
sample_df['LoanStatus'] = sample_df.LoanStatus.astype('category')
sample_df['BorrowerState'] = sample_df.BorrowerState.astype('category')
sample_df['Race'] = sample_df.Race.astype('category')

In [10]:
#Remove four last digit of zip
sample_df['BorrowerZip'] = sample_df['BorrowerZip'].str[:5]
sample_df['ProjectZip'] = sample_df['ProjectZip'].str[:5]
sample_df['ServicingLenderZip'] = sample_df['ServicingLenderZip'].str[:5]

In [ ]:
# Categorical consistency
rename_dict = {"Priority low":"low",
"priority_high":"high",
"priority Medium":"medium",
"priority_medium":"medium",
"priority low":"low"
}
sample_df["BusinessType"] = sample_df["BusinessType"].replace(rename_dict).astype("category")


In [11]:
# to  write the sampled file to drvie

sample_df.to_csv (r'/content/drive/Shared drives/DS4A_2021/Team74/data/Stage/sampled_data_stage.csv', index = None, header=True) 